In [1]:
# Import packages.
import cvxpy as cp
import numpy as np
import math
from scipy.linalg import expm
import matplotlib.pyplot as plt
import torch
from cvxpylayers.torch import CvxpyLayer

# Configure the display settings
np.set_printoptions(edgeitems=30, linewidth=125, 
    formatter=dict(float=lambda x: "%3.3f" % x))

In [2]:
# Parameter Initialization
# Background Parameter
g=[-3.7114, 0, 0]
pi = math.pi
m_dry=1505
m_wet=1905
I_sp=225
T1=930
T2=2480
n=6
phii=27*2*pi/360
alpha_val=6.293e-4
alpha = cp.Parameter(name="alpha", value=6.293e-4, nonneg=True)
#alphas = np.logspace(0, 0.01, 200)

In [3]:
# Optimal t_f and N
t_f=68
N=55
time_step=t_f/N
#Initial State
position_0=[3000, 1000, 2000]
velocity_0=[-50,10,100]
y0=np.array([position_0 + velocity_0 + [np.log(m_wet)]]).T
#print(y0, y0.shape)

In [4]:
# Define A_c Matrix
A_c1 = np.concatenate((np.zeros((3,3)), np.identity(3)), axis=1)
A_c2 = np.concatenate((A_c1, np.zeros((3,1))), axis=1)
A_c = np.concatenate((A_c2, np.zeros((4,7))), axis=0)
#print(A_c,A_c.shape)

In [5]:
# Define B_c Matrix
B_c1 = np.concatenate((np.zeros((3,3)), np.identity(3)),axis=0)
B_c2 = np.concatenate((B_c1, np.zeros((6,1))),axis=1)
B_c = np.concatenate((B_c2, np.array([[0,0,0,-alpha]])), axis=0)
print(B_c,B_c.shape)

[[0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0]
 [0.0 0.0 1.0 0.0]
 [0 0 0 Expression(CONSTANT, NONPOSITIVE, ())]] (7, 4)


In [6]:
# Define A & B Matrix
"""
  Work on this later!!!
  use SciPy
  
[A,B]=c2d(A_c,B_c,time_step);% A 7*7 ;B 7*4
"""



A = np.array([[1, 0, 0, 1.2364, 0, 0, 0],
              [0, 1, 0, 0, 1.2364, 0, 0],
              [0, 0, 1, 0, 0, 1.2364, 0],
              [0, 0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 1]

])
print("A=", A, A.shape)

B = np.array([[0.7643, 0, 0, 0],
          [0, 0.7643, 0, 0],
          [0, 0, 0.7643, 0],
          [1.2364, 0, 0, 0],
          [0, 1.2364, 0, 0],
          [0, 0, 1.2364, 0],
          [0, 0, 0, -alpha]])
print("B=", B, B.shape)
print(type(B))

A= [[1.000 0.000 0.000 1.236 0.000 0.000 0.000]
 [0.000 1.000 0.000 0.000 1.236 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 1.236 0.000]
 [0.000 0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 1.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 1.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 1.000]] (7, 7)
B= [[0.7643 0 0 0]
 [0 0.7643 0 0]
 [0 0 0.7643 0]
 [1.2364 0 0 0]
 [0 1.2364 0 0]
 [0 0 1.2364 0]
 [0 0 0 Expression(CONSTANT, NONPOSITIVE, ())]] (7, 4)
<class 'numpy.ndarray'>


In [7]:
psi = np.concatenate((B, np.zeros([7,N*4])),axis=1)
psi_prev = psi
for k in range(2,N+1):
    psi_next=A@psi_prev
    psi_next=np.concatenate((np.concatenate((psi_next[:,:4*(k-1)],B),axis=1),psi_next[:,4*(k-1)+4:]),axis=1)
    # psi_next[:-1,4*(k-1)-1] = 0 # clean up TODO: this is a hardcoded fix
    for i in range(k,1,-1):
        psi_next[:-1,4*(i-1)-1] = 0 # clean up TODO: this is a hardcoded fix
    psi = np.concatenate((psi, psi_next),axis=0)
    psi_prev = psi_next
print(psi.shape)

(385, 224)


In [8]:
# Matrix for State Computation  # A^k #(7*7N)
phi=np.zeros((7,7*N))
phi[:,0:7]=A
for k in range(2,N+1):
    phi[:,7*k-7:7*k]=A@phi[:,7*k-14:7*k-7]
print(phi.shape)

(7, 385)


In [9]:
# lambda matrix  #B+AB+......A^(k-1)B  #(7*4N)
lam=B  
lam_prev=B
for k in range(2,N+1):
    # print("k",k)
    lam_next=A@lam_prev+B
    lam_next[:-1,-1] = 0 # clean up
    lam = np.concatenate([lam, lam_next], axis=1)
    lam_prev = lam_next
lam.shape

(7, 220)

In [10]:
# upsilon matrix      #(4N*(4N+4))
upsilon=np.zeros((4*N,4*N+4)) 
for k in range(1,N+1):
    upsilon[4*k-4:4*k,4*k-4:4*k]=np.identity(4)
print(upsilon.shape)

(220, 224)


In [11]:
# Matrix for Optimal Computation 
k = 0
Z0 = np.array(cp.log(m_wet-alpha*n*T2*math.cos(phii)*time_step*k), ndmin=2)
for k in range(1,N+1):
    Z0_new = np.array(cp.log(m_wet-alpha*n*T2*math.cos(phii)*time_step*k), ndmin=2)
    Z0 = np.concatenate((Z0, Z0_new), axis=0)
print(Z0.shape)

(56, 1)


In [12]:
# Matrix for Optimal Computation 
mu_1 = np.array(n*T1*math.cos(phii)*cp.exp(-Z0[0,0]), ndmin = 2)  #(1*(N+1))
mu_2 = np.array(n*T2*math.cos(phii)*cp.exp(-Z0[0,0]), ndmin = 2)  #(1*(N+1))

for k in range(1, N+1):
    mu_1_new = np.array(n*T1*math.cos(phii)*cp.exp(-Z0[k,0]), ndmin = 2)
    mu_2_new = np.array(n*T2*math.cos(phii)*cp.exp(-Z0[k,0]), ndmin = 2)
    mu_1 = np.concatenate((mu_1, mu_1_new), axis = 1)
    mu_2 = np.concatenate((mu_2, mu_2_new), axis = 1)
print(mu_1.shape)
print(mu_2.shape)   

(1, 56)
(1, 56)


In [13]:
print(f'mu_1 shape = {mu_1.shape}')
print(f'mu_2 shape = {mu_2.shape}')

print(mu_1[0,:5], end="\n---------------------------------------\n")
print(mu_2[0,:5])

mu_1 shape = (1, 56)
mu_2 shape = (1, 56)
[Expression(CONSTANT, NONNEGATIVE, ()) Expression(CONSTANT, NONNEGATIVE, ()) Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ()) Expression(CONSTANT, NONNEGATIVE, ())]
---------------------------------------
[Expression(CONSTANT, NONNEGATIVE, ()) Expression(CONSTANT, NONNEGATIVE, ()) Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ()) Expression(CONSTANT, NONNEGATIVE, ())]


In [14]:
epsilon_k = phi[:,7*0:7*(0+1)]@y0+lam[:,4*0:4*(0+1)]@np.array([g + [0]]).T  #(7,1)
for k in range(1,N):
    temp = phi[:,7*k:7*(k+1)]@y0+lam[:,4*k:4*(k+1)]@np.array([g + [0]]).T
    epsilon_k = np.concatenate((epsilon_k, temp), axis = 1)
print(epsilon_k.shape)

(7, 55)


In [15]:
print(f'epsilon k shape = {epsilon_k.shape}')
print(epsilon_k[:,46])

epsilon k shape = (7, 55)
[-6171.981112688073 1581.1080000000006 7811.080000000007 -265.67242312000025 10.0 100.0
 Expression(CONSTANT, NONNEGATIVE, ())]


In [16]:
e_sigma = np.array([0,0,0,1])
#E = np.concatenate((np.identity(6), np.zeros((6,1))), axis=1)
F = np.concatenate((np.zeros(6), np.ones(1)), axis=None).reshape((1,7))
E_u = np.concatenate((np.identity(3), np.zeros((3,1))), axis=1)
print(F.shape)

(1, 7)


In [17]:
omega=np.zeros((1,4*N))  #1*4N
for k in range(0,N):
    omega[0,4*k:4*(k+1)]=e_sigma
omega=time_step*omega
print(omega.shape)

(1, 220)


In [18]:
# Define and solve the CVXPY problem.
eta = cp.Variable((4*N+4,1))
obj = cp.Minimize(omega@eta[0:4*N,0])

In [19]:
def multiply(X, Y):
    result = np.zeros((X.shape[0], Y.shape[1]))
    result = result.tolist()
    #print(f'a = {result}')
    # iterate through rows of X
    for i in range(X.shape[0]):
        # iterate through columns of Y
        for j in range(Y.shape[1]):
            # iterate through rows of Y
            for k in range(Y.shape[0]):
                result[i][j] += X[i][k] * Y[k][j]
    '''
    a = [[0]*mat2.shape[1]] * mat1.shape[0]
    for i in range(mat1.shape[0]):
        for j in range(mat2.shape[1]):
            for k in range(mat2.shape[0]):
                a[i][j] += mat1[i][k] * mat2[k][j]
    a = np.array(a)
    '''
    return np.array(result)

In [20]:
mat1 = np.array([[1, 0, 2],
                 [0, 1, 3]])
mat2 = np.array([[1, 0, 2, 4],
                 [5, 1, 3, 1],
                 [9, 1, 3, 7]])
print(mat1@mat2)
print(multiply(mat1, mat2))

[[19  2  8 18]
 [32  4 12 22]]
[[19.000 2.000 8.000 18.000]
 [32.000 4.000 12.000 22.000]]


In [21]:
def list_dot_product(v1, v2):
    v3 = 0
    for i in range(len(v1)):
        v3 += v1[i] * v2[i]
    return v3

In [22]:
#  Final height constraint
#h_const = [m@(epsilon_k[:,N-1]+psi[7*N-7:7*N,:]@eta) == np.zeros((7,1))]
m1 = np.concatenate((np.identity(3), np.zeros((3,4))), axis=1)
m2 = np.concatenate((np.zeros((4,3)), np.zeros((4,4))), axis=1)
m = np.concatenate((m1, m2), axis=0)
#print(f'm.shape = {m.shape}')
a = multiply(psi[7*N-7:7*N,:],eta)
#print(f'a.shape = {a.shape}')
b = epsilon_k[:,N-1, np.newaxis]+a
#print(f'b.shape = {b.shape}')
c = multiply(m,b)
#print(f'c.shape = {c.shape}')
h_const = [i == 0 for i in c[:,0]]
#print(h_const)

In [23]:
#  Final velocity constraint
a1 = np.expand_dims(epsilon_k[:,N-1],axis=1)
a2 = multiply(psi[7*N-7:7*N,:],eta)

v_const = [
    np.array([[0, 0, 0, 1, 0, 0, 0]])@cp.vstack((a1+a2).squeeze())==0,
    np.array([[0, 0, 0, 0, 1, 0, 0]])@cp.vstack((a1+a2).squeeze())==0,
    np.array([[0, 0, 0, 0, 0, 1, 0]])@cp.vstack((a1+a2).squeeze())==0,
]
v_const

[Equality(Expression(AFFINE, UNKNOWN, (1, 1)), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (1, 1)), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (1, 1)), Constant(CONSTANT, ZERO, ()))]

In [24]:
# Convexified Thrust Constraint
Conv_const = [
      cp.SOC(e_sigma@upsilon[4*k:4*k+4,:]@eta, E_u@upsilon[4*k:4*k+4,:]@eta) for k in range(N)
]

In [26]:
# thrust constraints
thr_const = [mu_1[0,0]*(1-(F@y0-Z0[0,0])+0.5*(F@y0-Z0[0,0])**2)<=e_sigma@upsilon[0:4,:]@eta,
            e_sigma@upsilon[0:4,:]@eta<=mu_2[0,0]*(1-(F@y0-Z0[0,0]))]

for k in range(0,1):
    print(f'thr_const for iter:{k}')
    h3 = multiply(psi[7*k:7*(k+1),:],eta) # good
    h1 = list_dot_product(F.squeeze(),(epsilon_k[:,k, np.newaxis]+h3)[:,0])-Z0[k+1,0]
    h = 1-h1+0.5*(h1**2)
    # h = 1-h1+0.5*(h1)
    xLHS = mu_1[0,k+1]*h
    xRHS = e_sigma@upsilon[4*k:4*k+4,:]@eta
    thr_const.append(xLHS<=xRHS)

    # yLHS = e_sigma@upsilon[4*k:4*k+4,:]@eta
    # yRHS = mu_2[0,k+1]*(1-h1)
    # thr_const.append(yLHS <= yRHS)
    

thr_const for iter:0


In [27]:
# # Fuel mass constraints
# f_const = []
# for k in range(0,N):
#     print(f'f_const for iter:{k}')
#     h3 = multiply(psi[7*k:7*(k+1),:],eta) # good
#     h1 = list_dot_product(F.squeeze(),(epsilon_k[:,k, np.newaxis]+h3)[:,0])
#     xLHS = h1
#     xRHS = cp.log(m_wet-alpha*n*T1*math.cos(phii)*time_step*(k+1))
#     f_const.append(xLHS <= xRHS)
    
#     yLHS = cp.log(m_wet-alpha*n*T2*math.cos(phii)*time_step*(k+1))
#     yRHS = h1
#     f_const.append(yLHS <= yRHS)

In [28]:
a1 = np.array([[0, 1, 0],[0, 0, 1]])@np.concatenate((np.identity(3),np.zeros((3,4))),axis=1)
a2 = np.expand_dims(epsilon_k[:,k],axis=1) + multiply(psi[7*(N-1):7*(N),:],eta)
a3 = multiply(a1,a2).squeeze()
a4 = cp.norm(cp.vstack(a3))
b1=np.array([[-0.268, 0, 0, 0, 0, 0, 0]])
b2=multiply(b1,a2) 
b2 = b2.tolist()[0][0]
cone_const = [(a4+b2) <= 0]
cone_const

[Inequality(Expression(CONVEX, UNKNOWN, ()))]

In [ ]:
socp_constraints =  Conv_const + h_const + v_const + cone_const #+ thr_const # + f_const 

prob = cp.Problem(obj, socp_constraints)
prob.solve()
print(obj.value)
print(prob.status)
print("Problem is dpp:",prob.is_dpp())
print("Problem is dcp:",prob.is_dcp())
#prob.solve()
#print(prob.parameters)

In [30]:
cvxpylayer = CvxpyLayer(prob, parameters=[alpha], variables=[eta])
alpha_tch = torch.randn(requires_grad=True)
solution = cvxpylayer(alpha_tch)
solution.backward()

#print(obj.value)
#print(prob.status)
#print(alpha.value)
print(alpha.gradient)

#prob.solve(requires_grad=True)
#prob.backward()              #backpropagate through solution
#print(alpha.gradient)


ValueError: Problem must be DPP.

In [ ]:
#     Output Information
# Converting output into manageable format
display_position = np.zeros((3,N+1))
display_velocity = np.zeros((3,N+1))
display_netthrust = np.zeros((3,N+1))
display_thrust = np.zeros((3,N+1))
display_allthrust = np.zeros((1,N+1))
display_allvelocity = np.zeros((1,N+1))
display_position[0:3, 0] = position_0
display_velocity[0:3, 0] = velocity_0
mass = np.zeros((1,N+1))
mass[0] = m_wet

In [ ]:

for k in range(0,N):
    v = (epsilon_k[:,k].reshape((7,1))+psi[7*k:7*(k+1),:]@eta.value)
    display_position[:,k+1] = (np.concatenate((np.identity(3), np.zeros((3,4))), axis=1)@v).reshape((3,))
    temp = np.concatenate((np.zeros((3,3)), np.identity(3)), axis=1)
    display_velocity[:,k+1] = (np.concatenate((temp, np.zeros((3,1))), axis=1)@v).reshape(3,)
    mass[0,k+1] = np.array([[0, 0, 0, 0, 0, 0, 1]])@v
    mass[0,k+1] = np.exp(mass[0,k+1])

for k in range(0,N):
    display_netthrust[:,k+1] = (np.concatenate((np.identity(3), np.zeros((3,1))), axis=1)@(upsilon[4*k:4*k+4,:]@eta.value)).reshape(3,)
    display_thrust[:,k+1] = display_netthrust[:,k+1]
    display_netthrust[:,k+1] = display_netthrust[:,k+1]*mass[0,k+1]
    #display_allthrust[0,k+1] = cp.norm(display_thrust[0:3,k+1],2)*mass[0,k+1]
    #display_allvelocity[0,k+1] = cp.norm(display_velocity[0:3,k+1],2)


In [ ]:
plt.plot(display_position[0,:],label="X")
plt.plot(display_position[1,:],label="Y")
plt.plot(display_position[2,:],label="Z")
plt.legend(loc="upper left")

In [ ]:
plt.plot(display_netthrust[0,:],label="X")
plt.plot(display_netthrust[1,:],label="Y")
plt.plot(display_netthrust[2,:],label="Z")
plt.legend(loc="upper left")

In [ ]:
m1 = plt.subplot(2,1,1)
m2 = plt.subplot(2,1,2)
m1.plot(mass[0,:],label="Z")

m2.plot(mass[0,50:52],label="Z")